# Proyecto de análisis de sentimientos con Python


Curso 2022/2023:
    Juan López Quirós
    Jose Ignacio Castro Vázquez

 Lo primero que hay que hacer es escoger o recopilar una primera versión de los datos necesarios que utilizarmos 
 para entrenar a nuestros modelos de Aprendizaje automático. 
 Tras haber estudiado los varios problemas con la API de twitter y las alternativas propuestas, nos decidimos por 
 buscar un dataset ya recopilado de tweets reales de la página web kaggle. En concreto nos decidimos por un dataset
 ya enfocado al análisis de sentimientos con más de 1.6 millones de tweets recopilados directamente de la API de twitter
 por lo que se ajusta perfectamente al proyecto.

 url : https://www.kaggle.com/datasets/kazanova/sentiment140

 Luego, tenemos que limpiar el dataset escogido para este proyecto.
 A nosotros solo nos interesa una columna en particular de todo el dataset y ese es la columna de texto, que contiene
 el contenido de los tweets en sí. Para leer el fichero de datos y procesarlo, utilizaremos pandas.

In [1]:
import pandas as pd

Definimos la función que se encarga de leer y procesar el fichero de datos. Esta función eliminará las columnas irrelevantes, seleccionará un subset de 5000 tweets que formará el corpus de nuestros modelos y por último los guardará en otro fichero para su uso más adelante.

In [2]:
def create_cleaned_csv(original_filename, target_filename):
    #uft-8 encoding didn't work, latin1 encoding did.
    data = pd.read_csv(original_filename, encoding="latin1", header=None)

    #We give the columns a name
    column_names = ["target", "id", "date", "flag", "user", "text"]
    data.columns = column_names

    #We eliminate innecessary columns
    data.drop(columns=["target", "id", "date", "flag", "user"], inplace=True)

    #We select a subset of 5000 tweets
    data = data[:5000]

    #Save data onto new file
    data.to_csv(target_filename, index=False)
    print("FINAL RESULT: \n")
    print(data)

In [3]:
create_cleaned_csv("tweet_data.csv", "subset_tweet_data.csv")        

FINAL RESULT: 

                                                   text
0     @switchfoot http://twitpic.com/2y1zl - Awww, t...
1     is upset that he can't update his Facebook by ...
2     @Kenichan I dived many times for the ball. Man...
3       my whole body feels itchy and like its on fire 
4     @nationwideclass no, it's not behaving at all....
...                                                 ...
4995                                    long day today 
4996                     a friend broke his promises.. 
4997               @gjarnling I am fine thanks - tired 
4998          trying to keep my eyes open..damn baking 
4999                        why the hell is it snowing 

[5000 rows x 1 columns]


A continuación, utilizamos la libreria NLTK para limpiar, tokenizar y lematizar nuestro dataset.

Para esto, descargamos los recursos necesarios para trabajar con la librería nltk.

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juanq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Para asegurarnos de que el recurso que contiene las stopwords se descargó correctamente, los mostramos por pantalla.

In [5]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{"couldn't", 'here', "it's", 'him', "wasn't", 'himself', 'have', 'myself', 'again', 'after', 'some', 'own', 'its', 'y', 'does', 'we', 'wasn', 'my', "wouldn't", 'at', 'what', 'few', "haven't", 'aren', 'doing', 'who', 'then', 'only', 'most', 'each', 'but', 'mustn', 'shouldn', 'while', 'is', 'herself', 'hers', 'your', 'are', 'that', 'too', 'nor', 'did', 'below', 'doesn', 'than', 'they', 'by', 'she', "should've", "weren't", 'any', 't', 'will', 'her', 'further', 'didn', 'he', 'over', 'because', 'down', 'weren', 'for', 'needn', 'out', 'whom', 'if', "shouldn't", 'into', 'all', 'ain', 'in', "isn't", 'yours', 'had', 'our', 'haven', 'mightn', "you'd", 'above', 'should', 'from', 'theirs', 'being', 'when', "needn't", 'until', 'before', 'm', 'or', 'you', 'wouldn', 'am', "you've", 'ours', 'these', 'as', 'not', 'this', 'were', 'themselves', 'yourself', 'which', 'same', 'don', 'why', 's', 'it', 'was', 'their', "doesn't", "hasn't", 'through', 'of', 'so', "you'll", "you're", 'yourselves', 'and', 'how', 

Definimos la función encargada de limpiar los tweets del dataset que eliminará las menciones, hashtags, urls y otros símbolos extraños contenidos en ellos.

Además después de pasar por este filtro, los tokenizamos con TweetTokenizer que a diferencia del tokenizador por defecto de NLTK, word_tokenizer, este si mantiene la coherencia en palabras complejas del inglés.

Por ejemplo la palabra "can't":
- Con word_tokenizer-> can't = ["ca", "n't"]
- Con TweetTokenize-> can't = ["can't"]

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from string import punctuation

def clean_text(original_filename, target_filename, with_punctuation=True):
    data = pd.read_csv(original_filename, encoding="latin1")
    stop_words = set(stopwords.words('english'))
    mention_or_hashtag_characters = ["@", "#"]

    tt = TweetTokenizer() #Tokenizer
    ps = PorterStemmer() #Stemmer

    for i, row in data.iterrows():
        tweet = data.iloc[i]["text"]

        tweet_tokens = tt.tokenize(tweet) #Tokenizing

        filtered_tweet_tokens = [token for token in tweet_tokens if 
                                    token.lower()[0] not in mention_or_hashtag_characters and #Eliminate mentions, hashtags
                                    token.lower() not in stop_words and #Eliminate stop words
                                    'http' not in token.lower()] #Eliminate urls

        if(not with_punctuation):
            filtered_tweet_tokens = [token for token in filtered_tweet_tokens if token not in list(punctuation)]

        stemmed_tweet_tokens = [ps.stem(token) for token in filtered_tweet_tokens] #Stemming
        
        data.loc[i,['text']] = ' '.join(str(token) for token in stemmed_tweet_tokens)

    data.to_csv(target_filename, index=False)

        


Hacemos 2 llamadas a la función. La primera para el corpus sea texto con signos de puntuación inclusive y otro con texto puramente alfanumérico, esto lo hacemos así para estudiar el posible impacto que puedan tener estos en los modelos finales. 

In [7]:
clean_text('subset_tweet_data.csv', 'tokenized_data.csv')
clean_text('subset_tweet_data.csv', 'tokenized_data_no_punctuation.csv', False)

El siguiente paso consisitirá en utilizar la librería TextBlob para la clasificacion de los tweets en: Muy Feliz, Contento, Neutro, Molesto, Hater.

Para ello primero debemos definir los términos y qué consideraremos como 'Neutro', 'Hater', etc..
TextBlob mediante su función 'TextBlob.sentiment' nos hace un análisis de sentimientos de cualquier texto en forma de Objeto Sentiment, un objeto que tiene 2 parámetros:

    - Polarity con rango [-1,1] que indica la polaridad del sentimiento del texto siendo -1 un texto muy negativo y 1 muy positivo
    
    - Subjectivity con rango [0,1] que indica la objetividad del texto siendo 0 muy objetivo y cercano a la realidad y 1 siendo muy subjetivo donde se expresan más las opiniones y sentimientos personales del autor.

Teniendo en cuenta las limitaciones del procesamiento de análsis de Textblob definiremos:

    - Muy Feliz: Polarity=(0.25,1] Subjectivity=[0-0.5)
    - Contento: Polarity=(0.25,1] Subjectivity=[0.5-1]
    - Neutro: Polarity=(-0.25,0.25] Subjectivity=any
    - Hater: Polarity=[-1,-0.25) Subjectivity=[0-0.5)
    - Molesto: Polarity=[-1,-0.25) Subjectivity=[0.5-1]
    
    Esto lo hacemos así ya que consideramos que la principal diferencia entre "Muy Feliz"-"Contento" y "Hater"-"Molesto" es que los primeros son opiniones más subjetivas que las segundas. Y por otra parte consideramos que si el texto es muy objetivo decimos que es neutro ya que no expresa sentimiento ninguno de la persona que escribe el tweet.

In [14]:
from textblob import TextBlob

def classify_tweets(original_filename, target_filename):
    data = pd.read_csv(original_filename, encoding="latin1")
    dic_count = dict()
    for i, row in data.iterrows():
        tweet = TextBlob(str(data.iloc[i]["text"]))
        sentiment = tweet.sentiment
        tag = choose_classification(sentiment)
        data.loc[i,['tag']] = tag

        if tag not in dic_count:
            dic_count[tag] = 1
        else:
            dic_count[tag] = dic_count[tag] + 1

    print(dic_count)
    data.to_csv(target_filename, index=False)


def choose_classification(sentiment):
    classification = "Neutral"
    polarity = sentiment.polarity 
    subjectivity = sentiment.subjectivity 
    if polarity < -0.25:
        classification = "Molesto"
        if subjectivity < 0.5:
            classification = "Hater"
    elif polarity > 0.25:
        classification = "Contento"
        if subjectivity < 0.5:
            classification = "Muy Feliz"
    return classification


Probamos la clasificación de los tweets en los 2 datasets uno con signos de puntuación y otro sin ellos. Podemos ver, por los diccionarios contadores calculados, que devuelven resultados muy parecidos.

In [17]:
classify_tweets("tokenized_data_no_punctuation.csv", "final_dataset.csv")
classify_tweets("tokenized_data.csv", "final_dataset_punctuation.csv")

{'Neutral': 3516, 'Molesto': 656, 'Contento': 596, 'Hater': 80, 'Muy Feliz': 152}
{'Neutral': 3467, 'Molesto': 667, 'Contento': 611, 'Hater': 83, 'Muy Feliz': 172}


In [5]:
# Basic math algorithms

'''def map_algorithm(candidates):

  max = {'probability': .0}

  for candidate in candidates:
    probability = candidate['probability']

    if(max['probability'] < probability):
      max = candidate

  return max'''

def basic_distance(tweet1, tweet2):

  distance = 0

  text1 = tweet1('text')
  text2 = tweet2('text')

  for word1 in text1:
    for word2 in text2:
      if word1 != word2:
        distance += 1
  
  return distance

def laplace(dividend, divisor, m = 1, k = 1):
  return (dividend + k) / (divisor + (k * m))

In [6]:
# Processing data

def probability_word_tagged(word, tag, tweets):

  n = 0
  m = 0

  for t in tweets:

    if t['tag'] == tag:
      m += 1

      if word in t['text']:
        n += 1

  return laplace(n, m, len(tweets))

def basic_voting(tweets):

  tag = ''
  i = 0

  tags = [t.get('tag') for t in tweets]

  for t in tuple(tags):
    n = tags.count(t)
    
    if i < n:
      i = n
      tag = t
  
  return tag

In [2]:
# Algorithms of machine learning

from itertools import islice

def basic_naive_bayes(tweet, tweets, c):

  max = 0
  tag = ''
  tags = [t.get('tag') for t in tweets]

  for tg in tuple(tags):
    prob_tagged = 1

    for word in tweet['text']:
      prob_tagged = prob_tagged * probability_word_tagged(word, tg, tweets)

    prob_tag = tags.count(tg)/len(tags)
    prob = prob_tag * prob_tagged

    if max < prob:
      tag = tg

  return tag

def basic_kNN(tweet, tweets, k):

  res = {}

  for text, tag in tweets:
    if text != tweet.get('text'):

      v = [tag, basic_distance(tweet, {text: tag})]
      res['text'] = v
  
  res = dict(sorted(res.items(), key=lambda item: item[1][1]))
  iterate_res = iter(res)

  kNN = list(islice(iterate_res, k))

  return basic_voting(kNN)